<a href="https://colab.research.google.com/github/OseiasBeu/Data-Science/blob/master/brincando_com_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [39]:
# Efetuando o dowload da base
!wget --show-progress https://raw.githubusercontent.com/OseiasBeu/Data-Science/master/DataSets/gamesSales.csv
!wget --show-progress https://raw.githubusercontent.com/OseiasBeu/Data-Science/master/DataSets/suicide_rates.csv

--2021-11-15 21:21:08--  https://raw.githubusercontent.com/OseiasBeu/Data-Science/master/DataSets/gamesSales.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1355781 (1.3M) [text/plain]
Saving to: ‘gamesSales.csv’

gamesSales.csv      100%[===================>]   1.29M  --.-KB/s    in 0.03s   

2021-11-15 21:21:09 (40.6 MB/s) - ‘gamesSales.csv’ saved [1355781/1355781]

--2021-11-15 21:21:09--  https://raw.githubusercontent.com/OseiasBeu/Data-Science/master/DataSets/suicide_rates.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Lengt

In [42]:
# iniciar uma sessão local e importar dados do Airbnb
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()
gameSales = spark.read.load('/content/gamesSales.csv',format='csv',inferSchema=True,header=True, sep=',')
gameSales.show()

+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+
|Rank|                Name|Platform|Year|       Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|
+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+
|   1|          Wii Sports|     Wii|2006|      Sports|            Nintendo|   41.49|   29.02|    3.77|       8.46|       82.74|
|   2|   Super Mario Bros.|     NES|1985|    Platform|            Nintendo|   29.08|    3.58|    6.81|       0.77|       40.24|
|   3|      Mario Kart Wii|     Wii|2008|      Racing|            Nintendo|   15.85|   12.88|    3.79|       3.31|       35.82|
|   4|   Wii Sports Resort|     Wii|2009|      Sports|            Nintendo|   15.75|   11.01|    3.28|       2.96|        33.0|
|   5|Pokemon Red/Pokem...|      GB|1996|Role-Playing|            Nintendo|   11.27|    8.89|   10.22|  

Verificando o schema inferido pelo spark

In [43]:
gameSales.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: double (nullable = true)
 |-- EU_Sales: double (nullable = true)
 |-- JP_Sales: double (nullable = true)
 |-- Other_Sales: double (nullable = true)
 |-- Global_Sales: double (nullable = true)



Convertendo coluna Year para integer

In [66]:
from pyspark.sql.types import IntegerType
gameSales = gameSales.withColumn('Year',gameSales['Year'].cast(IntegerType()))
gameSales.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: double (nullable = true)
 |-- EU_Sales: double (nullable = true)
 |-- JP_Sales: double (nullable = true)
 |-- Other_Sales: double (nullable = true)
 |-- Global_Sales: double (nullable = true)



In [24]:
suicides = spark.read.load('/content/suicide_rates.csv',inferSchema=True, header=True,format='csv',sep=',')
suicides.columns

['country',
 'year',
 'sex',
 'age',
 'suicides_no',
 'population',
 'suicides/100k pop',
 'country-year',
 'HDI for year',
 ' gdp_for_year ($) ',
 'gdp_per_capita ($)',
 'generation']

In [21]:
suicides.printSchema()

root
 |-- country: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- suicides_no: integer (nullable = true)
 |-- population: integer (nullable = true)
 |-- suicides/100k pop: double (nullable = true)
 |-- country-year: string (nullable = true)
 |-- HDI for year: double (nullable = true)
 |--  gdp_for_year ($) : string (nullable = true)
 |-- gdp_per_capita ($): integer (nullable = true)
 |-- generation: string (nullable = true)



Removendo a coluna country-year por ela não ter necessidade

In [25]:
suicides_columns = suicides.select('country', 'year', 'sex', 'age', 'suicides_no', 'population', 'suicides/100k pop', 'HDI for year', ' gdp_for_year ($) ', 'gdp_per_capita ($)', 'generation')
suicides_columns.show(5)

+-------+----+------+-----------+-----------+----------+-----------------+------------+------------------+------------------+---------------+
|country|year|   sex|        age|suicides_no|population|suicides/100k pop|HDI for year| gdp_for_year ($) |gdp_per_capita ($)|     generation|
+-------+----+------+-----------+-----------+----------+-----------------+------------+------------------+------------------+---------------+
|Albania|1987|  male|15-24 years|         21|    312900|             6.71|        null|     2,156,624,900|               796|   Generation X|
|Albania|1987|  male|35-54 years|         16|    308000|             5.19|        null|     2,156,624,900|               796|         Silent|
|Albania|1987|female|15-24 years|         14|    289700|             4.83|        null|     2,156,624,900|               796|   Generation X|
|Albania|1987|  male|  75+ years|          1|     21800|             4.59|        null|     2,156,624,900|               796|G.I. Generation|
|Alban

In [35]:
suicides2015 = suicides_columns.filter(suicides_columns['year']==2015)
suicides2017.show(5) 

+-------------------+----+------+-----------+-----------+----------+-----------------+------------+------------------+------------------+------------+
|            country|year|   sex|        age|suicides_no|population|suicides/100k pop|HDI for year| gdp_for_year ($) |gdp_per_capita ($)|  generation|
+-------------------+----+------+-----------+-----------+----------+-----------------+------------+------------------+------------------+------------+
|Antigua and Barbuda|2015|female|55-74 years|          1|      6403|            15.62|        null|     1,364,863,037|             14853|     Boomers|
|Antigua and Barbuda|2015|female|15-24 years|          0|      8561|              0.0|        null|     1,364,863,037|             14853|  Millenials|
|Antigua and Barbuda|2015|female|25-34 years|          0|      7740|              0.0|        null|     1,364,863,037|             14853|  Millenials|
|Antigua and Barbuda|2015|female|35-54 years|          0|     15323|              0.0|        

Capturando os anos maximos e minimos dos datasets:

In [68]:
max_suicides = suicides.agg({"year":"max"}).collect()[0]
min_suicides = suicides.agg({'year':'min'}).collect()[0]
max_salesGames = gameSales.agg({'Year':'max'}).collect()[0]
min_salesGames = gameSales.agg({'Year':'min'}).collect()[0]

In [69]:
print(f'''
    A data máxima de suicídios é de: {max_suicides[0]} \n
    A data minima de suicídios é de: {min_suicides[0]} \n
    A data maxima de vendas de videogames: {max_salesGames[0]} \n
    A data minima da venda de videogames: {min_salesGames[0]}
''')


    A data máxima de suicídios é de: 2016 

    A data minima de suicídios é de: 1985 

    A data maxima de vendas de videogames: 2020 

    A data minima da venda de videogames: 1980



In [75]:
gameSalesAjust = gameSales.filter((gameSales['Year']>=1985) & (gameSales['Year']<=2016))

Periodo Ajustado:

In [80]:
print(gameSalesAjust.agg({'Year':'max'}).collect()[0])
print(gameSalesAjust.agg({'Year':'min'}).collect()[0])


Row(max(Year)=2016)
Row(min(Year)=1985)


In [79]:
suicides.groupBy(suicides['country']).agg({'suicides_no':'sum'}).show()

+-----------------+----------------+
|          country|sum(suicides_no)|
+-----------------+----------------+
|         Paraguay|            4783|
|       Cabo Verde|              42|
|           Sweden|           37795|
|         Kiribati|              53|
|Republic of Korea|          261730|
|           Guyana|            3426|
|      Philippines|           21330|
|        Singapore|           10089|
|             Fiji|             304|
|           Turkey|           10131|
|          Germany|          291262|
|         Maldives|              20|
|           France|          329127|
|           Greece|           12368|
|        Sri Lanka|           55641|
|         Dominica|               0|
|         Slovakia|           13437|
|            Macau|              27|
|        Argentina|           82219|
|          Belgium|           62761|
+-----------------+----------------+
only showing top 20 rows

